Note: 
This is the exact set of code in Part A1a. But this one will mark the file as oncampus 

In [1]:
# THIS CODE WORKS AS WELL. MODIFED FROM CODE BY Petrina Collingwood


def domain_count(filename):
    
    import csv
    import re
    # create csv file from log file
    with open(filename,'r') as fh:
        with open('csv/' + filename + '.csv','w') as outfile:
            for line in fh:
                print(re.sub(r'\n|"','',line), file=outfile)
    import pandas as pd
    from urllib.parse import unquote
    # create dataframe from csv file skipping malformed lines
    df = pd.read_csv('csv/' + filename + '.csv',sep=' ', error_bad_lines=False, header=None, encoding='utf-8')
    # remove unnecessary columns
    df.drop(df.columns[[2,5,6,8,9]], axis=1, inplace=True)
    # name columns
    df.columns = ['ip', 'session_id', 'user_id', 'date_time', 'url', 'size']
    # formate date/time column
    df['date_time'] = df['date_time'].map(lambda x: x.lstrip('['))
    df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%b/%Y:%H:%M:%S')
    # remove lines where user is not logged in
    df = df[df.user_id != "-"]
    # decode urls
    def decode_url(url):
        decoded_url = unquote(url)
        return decoded_url
    df['url'] = df.url.apply(decode_url)
    # remove excess columns for domain
    df.drop(['ip','session_id','size'], axis=1, inplace=True)
    # remove ezp string from start of url
    df['url'] = df['url'].str.replace(r'^http://ezproxy\.lib\.ryerson\.ca/login/\?url=', '')# remove http etc
    df['url'] = df['url'].str.replace(r'^http://www\.|^https://www\.|^http://|^https://', '')
    # remove ezproxy string from start of url
    def parse_url(url):
        if (url.startswith("ezproxy.lib.ryerson.ca/login?url=")) and ("http" in url):
            location = url.find("http")
            return url[location:]
        elif (url.startswith("ezproxy.lib.ryerson.ca/login?url=")):
            return "-"
        else:
            return url
    df['url'] = df.url.apply(parse_url)
    # remove http etc
    df['url'] = df['url'].str.replace(r'^http://www\.|^https://www\.|^http://|^https://', '')
    # remove rows where ezproxy string is the only url
    df = df[df.url != "-"]
    # remove spaces introduced by unquoting
    df['url'] = df['url'].str.replace(r'\n', '')
    # remove everything after : or / or ?
    df['url'] = df['url'].str.replace(r'[:/?].*$', '')
    # remove .ezp.lib.unimelb.edu.au from urls
    df['url'] = df['url'].str.replace(r'ezproxy\.lib\.ryerson\.ca', '')
    df['url'] = df['url'].str.replace(r'ezproxy\.lib\.', '-')
    df = df[df.url != "-"]
    # create new column of domains
    def get_domain(url):
        regexp = re.compile(r'\.com|\.org|\.net|\.edu|-org|-com|\.gov')
        if regexp.search(url) is not None:
            for match in regexp.finditer(url):
                location = match.start()
            new_url = url[:location]
            if ('.' in new_url):
                location = new_url.rfind('.')
            elif ('-' in new_url):
                location = new_url.rfind('-')
            else:
                return url
            location += 1
            
            if ("-org" in url[location:]):
                modified_url = url[location:].replace(r'-org', '.org')
            elif ("-com" in url[location:]):
                modified_url = url[location:].replace(r'-com', '.com')
            else:
                return url[location:]
            return modified_url
        else:
            return url
    df['domain'] = df.url.apply(get_domain)
    
    
    # remove duplicate rows which have same user_id, date-time and domain. 
    df.drop_duplicates(subset=['date_time','domain'], inplace=True)

    df.to_csv('student_count.csv',index=False, encoding='utf-8')
    




A folder needs to be created in csv in order for this to work 

In [ ]:
#THIS CODE WORKS !!!
import pandas as pd
import glob
import os

studentdf = pd.read_csv("data/student1718.csv")
newdf=pd.DataFrame()

for filename in glob.iglob(os.path.join("oncampus/201808/","*.log")):
    filename = filename.replace("\\","/")
    # call log analysis function 
    domain_count(filename)
    # create dataframe from csv output file from daily domain count
    df=pd.read_csv("student_count.csv",sep=",")
    df_combined=pd.merge(df,studentdf,how="left",left_on="user_id",right_on="USERNAME")
    df_combined.columns=['user_id','date_time','url','domain','USERNAME',"acad_prog","prog_yr"]
    df_combined=df_combined.drop('USERNAME',axis=1)
    df_combined['acad_prog']=df_combined['acad_prog'].fillna("staff")
    df_combined['prog_yr']=df_combined['prog_yr'].fillna("Other")
    df_combined['on_or_off']="oncampus"
    newdf = newdf.append(df_combined)  
    newdf.to_csv('combined201808_oncampus.csv',index=False, encoding='utf-8')

In [65]:
df = pd.read_csv("combined201712_oncampus.csv",sep=",")
dfg=df.groupby(["domain","acad_prog"])['prog_yr'].count().unstack().fillna(0)
dfp=df.groupby(["domain","prog_yr"])['prog_yr'].count().unstack().fillna(0)
dfg.head()
dfp.head()

#the name of the file needs to be updated for each monthly file

dfg.to_excel("grouped_data_acadprog201712_oncampus.xls")
dfp.to_excel("group_data_progyr_201712oncampus.xls")
